In [32]:
import os
import pandas as pd
import sklearn as sk
import sklearn
from bs4 import BeautifulSoup
import re,string,unicodedata
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
#import the data
data = pd.read_csv("../res/Dataset.csv")
data.head(10)

print(data.shape)
print(data['sentiment'].value_counts())


(50000, 2)
positive    25000
negative    25000
Name: sentiment, dtype: int64


In [34]:
def cleanup(text):
    # Remove html tags
    text = BeautifulSoup(text, "html.parser").getText()
    # Remove square brackets 
    text = re.sub('\[[^]]*\]', '', text)
    # Remove special characters
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    # Remove Stopwords
    #text = text.lower().split()
    # Stemming
    ps = nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])

    return text
    
tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
data['review'] = data['review'].apply(cleanup)

C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [35]:
stops = nltk.corpus.stopwords.words("english")

#removing the stopwords
def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token.lower() not in stops]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
data['review'] = data['review'].apply(remove_stopwords)

In [36]:
#shuffle the data
data_norm_shuffle = data.sample(frac=1).reset_index(drop=True)

#split the data
split = 30000

train_reviews = data_norm_shuffle['review'][:split]
train_sentiments = data_norm_shuffle['sentiment'][:split]

test_reviews = data_norm_shuffle['review'][split:].reset_index(drop=True)
test_sentiments = data_norm_shuffle['sentiment'][split:].reset_index(drop=True)


print(train_reviews.shape, train_sentiments.shape)
print(train_reviews[0][0:50]," : ", train_sentiments[0])
print(test_reviews.shape, test_sentiments.shape)
print(test_reviews[0][0:50]," : ", test_sentiments[0])

(30000,) (30000,)
bought thi video walmart 1 bin think overpaid 1940  :  negative
(20000,) (20000,)
wonder cast thrown modern mystic romanc intellectu  :  positive


In [37]:
#Count vectorizer for bag of words
cv = sk.feature_extraction.text.CountVectorizer(min_df=0, max_df=1, binary=False, ngram_range=(1,3))
train_reviews_vector = cv.fit_transform(train_reviews)
test_reviews_vector = cv.transform(test_reviews)

print(train_reviews_vector.shape)
print(test_reviews_vector.shape)

(30000, 4822776)
(20000, 4822776)


In [38]:
#Tfidf vectorizer
tv = sk.feature_extraction.text.TfidfVectorizer(min_df=0, max_df=1, use_idf=True, ngram_range=(1,3))
train_reviews_tvector = tv.fit_transform(train_reviews)
test_reviews_tvector = tv.transform(test_reviews)

print(train_reviews_tvector.shape)
print(test_reviews_tvector.shape)

(30000, 4822776)
(20000, 4822776)


In [39]:
#vectorize the sentient data
lb = sk.preprocessing.LabelBinarizer()
train_sentiments_vector = lb.fit_transform(data_norm_shuffle['sentiment'])

#split the data
train_sentiments = train_sentiments_vector[:split]
test_sentiments = train_sentiments_vector[split:]
print(train_sentiments.shape)
print(test_sentiments.shape)

(30000, 1)
(20000, 1)


In [46]:
def learingmethod(string, model, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments):
    model.fit(train_reviews_vector, train_sentiments)
    predictions = model.predict(test_reviews_vector)
    print(string)
    print("Accuracy: ", sk.metrics.accuracy_score(test_sentiments, predictions))
    print("Precision: ", sk.metrics.precision_score(test_sentiments, predictions))
    print("Recall: ", sk.metrics.recall_score(test_sentiments, predictions))
    print("F1: ", sk.metrics.f1_score(test_sentiments, predictions))
    print("Confusion Matrix: ", sk.metrics.confusion_matrix(test_sentiments, predictions))
    print("Classification Report: ", sk.metrics.classification_report(test_sentiments, predictions))
    #write to file
    fileout = open("../res/" + string + ".txt", "w")
    with fileout as file:
        file.write(string + "\naccuracy_score: "  + str(sk.metrics.accuracy_score(test_sentiments, predictions)) + "\nprecision_score: " + str(sk.metrics.precision_score(test_sentiments, predictions)) + "\nrecall_score: " + str(sk.metrics.recall_score(test_sentiments, predictions)) + "\nf1_score: " + str(sk.metrics.f1_score(test_sentiments, predictions)) + "\nconfusion_matrix: "  + str(sk.metrics.confusion_matrix(test_sentiments, predictions)) + "\nclassification_report\n" + str(sk.metrics.classification_report(test_sentiments, predictions)))
    

In [42]:
lr = sk.linear_model.LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)
sgd = sk.linear_model.SGDClassifier(loss='hinge', max_iter=500, random_state=42)
br = sk.linear_model.BayesianRidge()
lars = sk.linear_model.Lars()
lasso = sk.linear_model.Lasso()
ridge = sk.linear_model.Ridge()
en = sk.linear_model.ElasticNet()
perceptron = sk.linear_model.Perceptron(max_iter=500, random_state=42)
passive_aggressive = sk.linear_model.PassiveAggressiveClassifier(max_iter=500, random_state=42)
svc = sk.svm.SVC(kernel='linear', C=1, random_state=42)
linear_svc = sk.svm.LinearSVC(max_iter=500, random_state=42)
nu_svc = sk.svm.NuSVC(kernel='linear', random_state=42)

In [47]:
learingmethod("LinearRegression BOW", lr, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("LinearRegression TFIDF", lr, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearRegression BOW
Accuracy:  0.59865
Precision:  0.9017964071856287
Recall:  0.225
F1:  0.3601434834595456
Confusion Matrix:  [[9714  246]
 [7781 2259]]
Classification Report:                precision    recall  f1-score   support

           0       0.56      0.98      0.71      9960
           1       0.90      0.23      0.36     10040

    accuracy                           0.60     20000
   macro avg       0.73      0.60      0.53     20000
weighted avg       0.73      0.60      0.53     20000



C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearRegression TFIDF
Accuracy:  0.7395
Precision:  0.7681545636242505
Recall:  0.6890438247011952
F1:  0.7264517483986139
Confusion Matrix:  [[7872 2088]
 [3122 6918]]
Classification Report:                precision    recall  f1-score   support

           0       0.72      0.79      0.75      9960
           1       0.77      0.69      0.73     10040

    accuracy                           0.74     20000
   macro avg       0.74      0.74      0.74     20000
weighted avg       0.74      0.74      0.74     20000



In [50]:
learingmethod("SGDClassifier BOW", sgd, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("SGDClassifier TFIDF", sgd, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier BOW
Accuracy:  0.5319
Precision:  0.9568733153638814
Recall:  0.07071713147410359
F1:  0.13170098312001483
Confusion Matrix:  [[9928   32]
 [9330  710]]
Classification Report:                precision    recall  f1-score   support

           0       0.52      1.00      0.68      9960
           1       0.96      0.07      0.13     10040

    accuracy                           0.53     20000
   macro avg       0.74      0.53      0.41     20000
weighted avg       0.74      0.53      0.40     20000



C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier TFIDF
Accuracy:  0.498
Precision:  0.0
Recall:  0.0
F1:  0.0
Confusion Matrix:  [[ 9960     0]
 [10040     0]]
Classification Report:                precision    recall  f1-score   support

           0       0.50      1.00      0.66      9960
           1       0.00      0.00      0.00     10040

    accuracy                           0.50     20000
   macro avg       0.25      0.50      0.33     20000
weighted avg       0.25      0.50      0.33     20000



C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Mat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precisi

In [51]:
learingmethod("BayesianRidge BOW", br, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("BayesianRidge TFIDF", br, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [52]:
learingmethod("Lars BOW", lars, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("Lars TFIDF", lars, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [53]:
learingmethod("Lasso BOW", lasso, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("Lasso TFIDF", lasso, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

Lasso BOW


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
learingmethod("Ridge BOW", ridge, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("Ridge TFIDF", ridge, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("ElasticNet BOW", en, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("ElasticNet TFIDF", en, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("Perceptron BOW", perceptron, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("Perceptron TFIDF", perceptron, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("PassiveAggressiveClassifier BOW", passive_aggressive, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("PassiveAggressiveClassifier TFIDF", passive_aggressive, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("SVC BOW", svc, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("SVC TFIDF", svc, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("LinearSVC BOW", linear_svc, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("LinearSVC TFIDF", linear_svc, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("NuSVC BOW", nu_svc, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("NuSVC TFIDF", nu_svc, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
gnb = sk.naive_bayes.GaussianNB()
bnb = sk.naive_bayes.BernoulliNB()
mnb = sk.naive_bayes.MultinomialNB()
conb = sk.naive_bayes.ComplementNB()
canb = sk.naive_bayes.CategoricalNB()

In [ ]:
learingmethod("GaussianNB BOW", gnb, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("GaussianNB TFIDF", gnb, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("BernoulliNB BOW", bnb, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("BernoulliNB TFIDF", bnb, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("MultinomialNB BOW", mnb, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("MultinomialNB TFIDF", mnb, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("ComplementNB BOW", conb, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("ComplementNB TFIDF", conb, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)

In [ ]:
learingmethod("CategoricalNB BOW", canb, train_reviews_vector, train_sentiments, test_reviews_vector, test_sentiments)
learingmethod("CategoricalNB TFIDF", canb, train_reviews_tvector, train_sentiments, test_reviews_tvector, test_sentiments)